In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_TRACING_v2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "tutorial"
os.environ["LANGSMITH_API_KEY"] = ""

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

True

In [3]:
from langsmith import utils
utils.tracing_is_enabled()

True

In [4]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=os.getenv("LLM_MODEL"))

In [7]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = model.bind_tools([multiply])

In [8]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")

In [9]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': '94fad2a0-9978-416b-a412-9ca72f80f348',
  'type': 'tool_call'}]

In [10]:
result

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-17T14:46:03.352120841Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4233483567, 'load_duration': 3778663313, 'prompt_eval_count': 187, 'prompt_eval_duration': 130000000, 'eval_count': 22, 'eval_duration': 322000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-c9a44fc0-2aaa-4930-8277-8621d37ed4c4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': '94fad2a0-9978-416b-a412-9ca72f80f348', 'type': 'tool_call'}], usage_metadata={'input_tokens': 187, 'output_tokens': 22, 'total_tokens': 209})

In [12]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = model.bind_tools(tools)

In [13]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'e6b7ecfe-83b7-4521-ab86-1a0d9383278d', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '86819db4-88b7-4b44-b13a-02104beb4a12', 'type': 'tool_call'}]


In [14]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-17T14:50:18.322003455Z', 'done': True, 'done_reason': 'stop', 'total_duration': 666838547, 'load_duration': 18874800, 'prompt_eval_count': 231, 'prompt_eval_duration': 13000000, 'eval_count': 44, 'eval_duration': 633000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-daca7adf-67a9-4d57-b23e-c79ba3492c93-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'e6b7ecfe-83b7-4521-ab86-1a0d9383278d', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '86819db4-88b7-4b44-b13a-02104beb4a12', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens': 44, 'total_tokens': 275}),
 ToolMessage(content='36', name='multiply', tool_call_id='e6b7ecfe-83b7-4521-ab86-1a0d9383278d'),
 ToolMess

In [18]:
query = "What is 3 * 12? Also, what is 11 + 49?"
messages = [HumanMessage(query)]

# Invoke the LLM with tool use
ai_msg = llm_with_tools.invoke(messages)

# Process each tool call
for tool_call in ai_msg.tool_calls:
    operation = tool_call["name"].lower()
    selected_tool = {"add": add, "multiply": multiply}[operation]
    
    tool_msg = selected_tool.invoke(tool_call)
    
    # Print result in a human-readable way
    print(f"The result from the {operation} operation is: {tool_msg.content}")

The result from the multiply operation is: 36
The result from the add operation is: 60


In [21]:
from langchain_core.messages import ToolMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

# First, ask the model
ai_msg = llm_with_tools.invoke(messages)

# Collect tool responses
tool_responses = []
for tool_call in ai_msg.tool_calls:
    operation = tool_call["name"].lower()
    selected_tool = {"add": add, "multiply": multiply}[operation]
    tool_msg = selected_tool.invoke(tool_call)
    tool_responses.append(tool_msg)

# Append original AI message + tool responses
messages.append(ai_msg)
messages.extend(tool_responses)

# Let the model now "wrap it up" in a human-friendly answer
final_response = llm_with_tools.invoke(messages)

print(final_response.content)

The result of 3 * 12 is 36.

The result of 11 + 49 is 60.
